In [ ]:
import pandas as pd 

def evaluate_cumulative_cost(data,maker_fee=0.001,taker_fee=0.002):
    
    """
    Evaluate the cumulative cost of a trading strategy.

    Parameters:
    - data: pandas DataFrame with columns:
        'open', 'high', 'low', 'close', 'volume', 'signal', 'funding_rate'
        The 'signal' column indicates the desired position:
            1: long
            -1: short
            0: no position (i.e., close the current position)
            
    - maker_fee: maker fee rate (e.g., 0.0002 for 0.02%)
    - taker_fee: taker fee rate (e.g., 0.0004 for 0.04%)

    Returns:
    - result_df: pandas DataFrame with columns:
        'fee', 'funding_payment', 'cumulative_cost'
    """
    
    cumulative_cost = 0
    trade_cost = []
    funding_cost = []
    cumulative_cost = []
    position = 0
    
    for idx , row in data.iterrows() :
        
        next_position = row['signal']
        
        price = row['Close']
        
        funding_rate = row['funding_rate']
        
        # if the position changes, calculate the trade cost
        if next_position != position :
            
            size = next_position - position
            
            trade_notional = abs(size) * price
            
    
            if size > 0 :
                # long position
                trade_cost = trade_notional * maker_fee
            else:
                # short position
                trade_cost = trade_notional * taker_fee
        
        else:
            trade_cost = 0
        
        position = next_position
        
        # if the position is not 0, calculate the funding cost
        if position != 0 and funding_rate is not None:
            
            cumulative_cost += abs(position) * price * funding_rate
        
        else:
            funding_cost = 0
            
        cumulative_cost += trade_cost + funding_cost
        
        trade_cost.append(trade_cost)
        funding_cost.append(funding_cost)
        cumulative_cost.append(cumulative_cost)
        
    result_df = pd.DataFrame({'trade_cost':trade_cost,
                                'funding_cost':funding_cost,
                                'cumulative_cost':cumulative_cost}
                             , index = data.index)
    
    return result_df
    # Test Test Ricky